In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 16.9 MB/s eta 0:00:00


In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from catboost import CatBoostClassifier, Pool
import optuna
import warnings
from google.colab import drive

# *** ИЗМЕНЕНИЕ: Используем SMOTETomek ***
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE # SMOTETomek требует SMOTE

# Настройка логирования
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore", category=UserWarning)

# Монтирование Google Drive
drive.mount('/content/drive')

# Загрузка данных
train = pd.read_csv('/content/drive/My Drive/train.csv')
test = pd.read_csv('/content/drive/My Drive/test.csv')
submission = pd.read_csv('/content/drive/My Drive/sample_submission.csv')

print("Размерность тренировочных данных:", train.shape)
print("Размерность тестовых данных:", test.shape)
print("Распределение классов в тренировочных данных:\n", train['Churn'].value_counts())

# --- 2. Улучшенная Предобработка и Генерация Признаков (Feature Engineering) ---
def enhance_features(dataframe):
    df_enhanced = dataframe.copy()

    # Преобразование TotalCharges
    df_enhanced['TotalCharges'] = pd.to_numeric(df_enhanced['TotalCharges'], errors='coerce')
    df_enhanced['TotalCharges'] = df_enhanced['TotalCharges'].fillna(df_enhanced['TotalCharges'].median())

    # Вычисление AvgMonthlyCharge
    df_enhanced['AvgMonthlyCharge'] = df_enhanced['TotalCharges'] / df_enhanced['tenure'].replace(0, 1)

    # Базовые новые признаки (оставлены проверенные, без сильно сложных взаимодействий)
    df_enhanced['HasStreaming'] = ((df_enhanced['StreamingTV'] == 'Yes') | (df_enhanced['StreamingMovies'] == 'Yes')).astype(int)
    df_enhanced['HasMultipleServices'] = ((df_enhanced['OnlineSecurity'] == 'Yes') |
                                         (df_enhanced['OnlineBackup'] == 'Yes') |
                                         (df_enhanced['DeviceProtection'] == 'Yes') |
                                         (df_enhanced['TechSupport'] == 'Yes')).astype(int)
    df_enhanced['TenureMonthlyInteraction'] = df_enhanced['tenure'] * df_enhanced['MonthlyCharges']
    df_enhanced['IsLongTerm'] = (df_enhanced['tenure'] > 24).astype(int)
    df_enhanced['NoInternetService'] = (df_enhanced['InternetService'] == 'No').astype(int)
    df_enhanced['NoPhoneService'] = (df_enhanced['PhoneService'] == 'No').astype(int)
    df_enhanced['FiberOptic'] = (df_enhanced['InternetService'] == 'Fiber optic').astype(int)

    # Список числовых признаков для масштабирования
    numerical_features_for_scaling = [
        'tenure', 'MonthlyCharges', 'TotalCharges', 'AvgMonthlyCharge',
        'TenureMonthlyInteraction', 'HasStreaming', 'HasMultipleServices',
        'IsLongTerm', 'NoInternetService', 'NoPhoneService', 'FiberOptic'
    ]
    existing_numerical_features = [f for f in numerical_features_for_scaling if f in df_enhanced.columns and pd.api.types.is_numeric_dtype(df_enhanced[f])]

    if existing_numerical_features:
        scaler = StandardScaler()
        df_enhanced[existing_numerical_features] = scaler.fit_transform(df_enhanced[existing_numerical_features])

    # Преобразование Churn (только для тренировочных данных)
    if 'Churn' in df_enhanced.columns:
        df_enhanced['Churn'] = df_enhanced['Churn'].fillna('No')
        df_enhanced['Churn'] = df_enhanced['Churn'].map({'Yes': 1, 'No': 0}).astype('int64')

    # Преобразование gender
    if 'gender' in df_enhanced.columns:
        df_enhanced['gender'] = df_enhanced['gender'].map({'Male': 0, 'Female': 1}).astype('int64')

    return df_enhanced


# Вызов функции enhance_features для train и test
train_processed = enhance_features(train.copy())
test_processed = enhance_features(test.copy())


# --- 3. Подготовка данных для обучения и тестирования ---

if 'Churn' not in train_processed.columns:
    raise ValueError("Столбец Churn отсутствует после предобработки.")

X = train_processed.drop(['id', 'customerID', 'Churn'], axis=1, errors='ignore')
y = train_processed['Churn'].copy()

X = pd.get_dummies(X, drop_first=True)

X_test_final = test_processed.drop(['id', 'customerID'], axis=1, errors='ignore')
X_test_final = pd.get_dummies(X_test_final, drop_first=True)

train_cols = X.columns
X_test_final = X_test_final.reindex(columns=train_cols, fill_value=0)


# --- 4. Оптимизация Гиперпараметров с Optuna для CatBoost (SMOTETomek внутри K-Fold) ---
def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 600, 2500),
        'depth': trial.suggest_int('depth', 4, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 5.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 30),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_seed': 42,
        'verbose': 0,
        'eval_metric': 'F1',
        'early_stopping_rounds': 100,
    }

    # *** ИЗМЕНЕНИЕ: Оптимизируем sampling_strategy и n_neighbors для SMOTE внутри SMOTETomek ***
    sampling_strategy_smote = trial.suggest_float('sampling_strategy_smote', 0.6, 0.95)
    n_neighbors_smote = trial.suggest_int('n_neighbors_smote', 3, 10) # n_neighbors для SMOTE

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # Можно попробовать n_splits=7 или 10
    f1_scores = []

    for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
        y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

        # *** ИЗМЕНЕНИЕ: Используем SMOTETomek ***
        # Передаем smote_sampler с оптимизированными параметрами
        smotetomek_sampler_fold = SMOTETomek(
            random_state=42,
            smote=SMOTE(sampling_strategy=sampling_strategy_smote, random_state=42, k_neighbors=n_neighbors_smote)
        )
        X_train_resampled, y_train_resampled = smotetomek_sampler_fold.fit_resample(X_train_fold, y_train_fold)

        model = CatBoostClassifier(**params)
        model.fit(X_train_resampled, y_train_resampled, eval_set=(X_val_fold, y_val_fold), use_best_model=True)

        y_pred_val = model.predict(X_val_fold)
        f1_scores.append(f1_score(y_val_fold, y_pred_val, zero_division=0))

    return np.mean(f1_scores)

print("\n--- Запуск оптимизации гиперпараметров для CatBoost с Optuna ---")
study_catboost = optuna.create_study(direction='maximize', study_name='catboost_f1_optimization')
study_catboost.optimize(objective_catboost, n_trials=200) # Увеличим до 200 трайлов, если позволяет время

print("\nЛучшая попытка CatBoost (по Optuna):")
print(f"  F1-мера (средняя по CV): {study_catboost.best_value:.4f}")
print("  Лучшие гиперпараметры CatBoost:")
best_catboost_params = study_catboost.best_params
for key, value in best_catboost_params.items():
    print(f"    {key}: {value}")


# --- 5. Обучение финальной модели CatBoost с лучшими гиперпараметрами ---
print("\n--- Обучение финальной модели CatBoost на всех сбалансированных тренировочных данных ---")

# Извлекаем параметры SMOTETomek из best_catboost_params
final_sampling_strategy_smote = best_catboost_params.pop('sampling_strategy_smote', 0.65)
final_n_neighbors_smote = best_catboost_params.pop('n_neighbors_smote', 5)

# *** ИЗМЕНЕНИЕ: Используем SMOTETomek для финальной модели ***
smotetomek_sampler_final = SMOTETomek(
    random_state=42,
    smote=SMOTE(sampling_strategy=final_sampling_strategy_smote, random_state=42, k_neighbors=final_n_neighbors_smote)
)
X_final_resampled, y_final_resampled = smotetomek_sampler_final.fit_resample(X, y)

final_catboost_model = CatBoostClassifier(**best_catboost_params)
final_catboost_model.fit(X_final_resampled, y_final_resampled, verbose=0)


# --- 6. Оценка финальной модели на ОТДЕЛЬНОЙ ВАЛИДАЦИОННОЙ выборке для подбора порога ---
X_train_eval, X_val_eval, y_train_eval, y_val_eval = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\n--- Оценка финальной модели CatBoost на валидационной выборке для подбора порога ---")
y_pred_proba_val = final_catboost_model.predict_proba(X_val_eval)[:, 1]

best_f1_threshold = 0
best_threshold = 0.5
precision_at_best_threshold = 0
recall_at_best_threshold = 0

for threshold in np.arange(0.05, 0.96, 0.01):
    y_pred_tuned = (y_pred_proba_val >= threshold).astype(int)
    current_f1 = f1_score(y_val_eval, y_pred_tuned, zero_division=0)
    if current_f1 > best_f1_threshold:
        best_f1_threshold = current_f1
        best_threshold = threshold
        precision_at_best_threshold = precision_score(y_val_eval, y_pred_tuned, zero_division=0)
        recall_at_best_threshold = recall_score(y_val_eval, y_pred_tuned, zero_division=0)

print(f"Оптимальный порог для F1 на валидации: {best_threshold:.2f}")
print(f"Итоговая F1-мера на валидации (с оптимальным порогом): {best_f1_threshold:.4f}")
print(f"Precision при оптимальном пороге: {precision_at_best_threshold:.4f}")
print(f"Recall при оптимальном пороге: {recall_at_best_threshold:.4f}")


# --- 7. Предсказания для submission ---
print("\n--- Предсказания для submission ---")
test_preds_proba = final_catboost_model.predict_proba(X_test_final)[:, 1]

test_preds_tuned = (test_preds_proba >= best_threshold).astype(int)
submission['Churn'] = np.where(test_preds_tuned == 1, 'Yes', 'No')

print("\nПроверка структуры submission:")
print("Столбцы:", submission.columns.tolist())
print("Первые 5 строк перед сохранением:")
print(submission.head())

submission = submission[['id', 'Churn']]

submission.to_csv('submission_catboost_smotetomek_exp.csv', index=False)
print("Файл submission_catboost_smotetomek_exp.csv сохранен локально.")

Mounted at /content/drive
Размерность тренировочных данных: (5635, 20)
Размерность тестовых данных: (1408, 19)
Распределение классов в тренировочных данных:
 Churn
No     4138
Yes    1497
Name: count, dtype: int64

--- Запуск оптимизации гиперпараметров для CatBoost с Optuna ---

Лучшая попытка CatBoost (по Optuna):
  F1-мера (средняя по CV): 0.6357
  Лучшие гиперпараметры CatBoost:
    iterations: 1191
    depth: 4
    learning_rate: 0.12123202899967368
    l2_leaf_reg: 0.020965132498990433
    subsample: 0.6444246757419851
    colsample_bylevel: 0.799017582927282
    min_data_in_leaf: 11
    border_count: 116
    sampling_strategy_smote: 0.7770317150610612
    n_neighbors_smote: 9

--- Обучение финальной модели CatBoost на всех сбалансированных тренировочных данных ---

--- Оценка финальной модели CatBoost на валидационной выборке для подбора порога ---
Оптимальный порог для F1 на валидации: 0.48
Итоговая F1-мера на валидации (с оптимальным порогом): 0.8471
Precision при оптимальном 

In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from catboost import CatBoostClassifier, Pool
import optuna
import warnings
from google.colab import drive

# *** ИЗМЕНЕНИЕ: Используем SMOTEENN ***
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE # SMOTEENN требует SMOTE

# Настройка логирования
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore", category=UserWarning)

# Монтирование Google Drive
drive.mount('/content/drive')

# Загрузка данных
train = pd.read_csv('/content/drive/My Drive/train.csv')
test = pd.read_csv('/content/drive/My Drive/test.csv')
submission = pd.read_csv('/content/drive/My Drive/sample_submission.csv')

print("Размерность тренировочных данных:", train.shape)
print("Размерность тестовых данных:", test.shape)
print("Распределение классов в тренировочных данных:\n", train['Churn'].value_counts())

# --- 2. Улучшенная Предобработка и Генерация Признаков (Feature Engineering) ---
def enhance_features(dataframe):
    df_enhanced = dataframe.copy()

    # Преобразование TotalCharges
    df_enhanced['TotalCharges'] = pd.to_numeric(df_enhanced['TotalCharges'], errors='coerce')
    df_enhanced['TotalCharges'] = df_enhanced['TotalCharges'].fillna(df_enhanced['TotalCharges'].median())

    # Вычисление AvgMonthlyCharge
    df_enhanced['AvgMonthlyCharge'] = df_enhanced['TotalCharges'] / df_enhanced['tenure'].replace(0, 1)

    # Базовые новые признаки (оставлены проверенные, без сильно сложных взаимодействий)
    df_enhanced['HasStreaming'] = ((df_enhanced['StreamingTV'] == 'Yes') | (df_enhanced['StreamingMovies'] == 'Yes')).astype(int)
    df_enhanced['HasMultipleServices'] = ((df_enhanced['OnlineSecurity'] == 'Yes') |
                                         (df_enhanced['OnlineBackup'] == 'Yes') |
                                         (df_enhanced['DeviceProtection'] == 'Yes') |
                                         (df_enhanced['TechSupport'] == 'Yes')).astype(int)
    df_enhanced['TenureMonthlyInteraction'] = df_enhanced['tenure'] * df_enhanced['MonthlyCharges']
    df_enhanced['IsLongTerm'] = (df_enhanced['tenure'] > 24).astype(int)
    df_enhanced['NoInternetService'] = (df_enhanced['InternetService'] == 'No').astype(int)
    df_enhanced['NoPhoneService'] = (df_enhanced['PhoneService'] == 'No').astype(int)
    df_enhanced['FiberOptic'] = (df_enhanced['InternetService'] == 'Fiber optic').astype(int)

    # Список числовых признаков для масштабирования
    numerical_features_for_scaling = [
        'tenure', 'MonthlyCharges', 'TotalCharges', 'AvgMonthlyCharge',
        'TenureMonthlyInteraction', 'HasStreaming', 'HasMultipleServices',
        'IsLongTerm', 'NoInternetService', 'NoPhoneService', 'FiberOptic'
    ]
    existing_numerical_features = [f for f in numerical_features_for_scaling if f in df_enhanced.columns and pd.api.types.is_numeric_dtype(df_enhanced[f])]

    if existing_numerical_features:
        scaler = StandardScaler()
        df_enhanced[existing_numerical_features] = scaler.fit_transform(df_enhanced[existing_numerical_features])

    # Преобразование Churn (только для тренировочных данных)
    if 'Churn' in df_enhanced.columns:
        df_enhanced['Churn'] = df_enhanced['Churn'].fillna('No')
        df_enhanced['Churn'] = df_enhanced['Churn'].map({'Yes': 1, 'No': 0}).astype('int64')

    # Преобразование gender
    if 'gender' in df_enhanced.columns:
        df_enhanced['gender'] = df_enhanced['gender'].map({'Male': 0, 'Female': 1}).astype('int64')

    return df_enhanced


# Вызов функции enhance_features для train и test
train_processed = enhance_features(train.copy())
test_processed = enhance_features(test.copy())


# --- 3. Подготовка данных для обучения и тестирования ---

if 'Churn' not in train_processed.columns:
    raise ValueError("Столбец Churn отсутствует после предобработки.")

X = train_processed.drop(['id', 'customerID', 'Churn'], axis=1, errors='ignore')
y = train_processed['Churn'].copy()

X = pd.get_dummies(X, drop_first=True)

X_test_final = test_processed.drop(['id', 'customerID'], axis=1, errors='ignore')
X_test_final = pd.get_dummies(X_test_final, drop_first=True)

train_cols = X.columns
X_test_final = X_test_final.reindex(columns=train_cols, fill_value=0)


# --- 4. Оптимизация Гиперпараметров с Optuna для CatBoost (SMOTEENN внутри K-Fold) ---
def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 600, 2500),
        'depth': trial.suggest_int('depth', 4, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 5.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.6, 1.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 30),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_seed': 42,
        'verbose': 0,
        'eval_metric': 'F1',
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 50, 150), # Оптимизируем early_stopping_rounds
        # *** ИЗМЕНЕНИЕ: Добавляем scale_pos_weight ***
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 5.0),
    }


    sampling_strategy_smote = trial.suggest_float('sampling_strategy_smote', 0.6, 0.95)
    n_neighbors_smote = trial.suggest_int('n_neighbors_smote', 3, 10) # n_neighbors для SMOTE

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # Можно попробовать n_splits=7 или 10
    f1_scores = []

    for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
        y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

        # *** ИЗМЕНЕНИЕ: Используем SMOTEENN ***
        # Передаем smote_sampler с оптимизированными параметрами k_neighbors
        smoteenn_sampler_fold = SMOTEENN(
            random_state=42,
            smote=SMOTE(sampling_strategy=sampling_strategy_smote, random_state=42, k_neighbors=n_neighbors_smote)
        )
        X_train_resampled, y_train_resampled = smoteenn_sampler_fold.fit_resample(X_train_fold, y_train_fold)

        model = CatBoostClassifier(**params)
        model.fit(X_train_resampled, y_train_resampled, eval_set=(X_val_fold, y_val_fold), use_best_model=True)

        y_pred_val = model.predict(X_val_fold)
        f1_scores.append(f1_score(y_val_fold, y_pred_val, zero_division=0))

    return np.mean(f1_scores)

print("\n--- Запуск оптимизации гиперпараметров для CatBoost с Optuna ---")
study_catboost = optuna.create_study(direction='maximize', study_name='catboost_f1_optimization')
study_catboost.optimize(objective_catboost, n_trials=200) # Повысим до 200 трайлов, если позволяет время

print("\nЛучшая попытка CatBoost (по Optuna):")
print(f"  F1-мера (средняя по CV): {study_catboost.best_value:.4f}")
print("  Лучшие гиперпараметры CatBoost:")
best_catboost_params = study_catboost.best_params
for key, value in best_catboost_params.items():
    print(f"    {key}: {value}")


# --- 5. Обучение финальной модели CatBoost с лучшими гиперпараметрами ---
print("\n--- Обучение финальной модели CatBoost на всех сбалансированных тренировочных данных ---")

# Извлекаем параметры SMOTEENN из best_catboost_params
final_sampling_strategy_smote = best_catboost_params.pop('sampling_strategy_smote', 0.65)
final_n_neighbors_smote = best_catboost_params.pop('n_neighbors_smote', 5)

# *** ИЗМЕНЕНИЕ: Используем SMOTEENN для финальной модели ***
smoteenn_sampler_final = SMOTEENN(
    random_state=42,
    smote=SMOTE(sampling_strategy=final_sampling_strategy_smote, random_state=42, k_neighbors=final_n_neighbors_smote)
)
X_final_resampled, y_final_resampled = smoteenn_sampler_final.fit_resample(X, y)

final_catboost_model = CatBoostClassifier(**best_catboost_params)
final_catboost_model.fit(X_final_resampled, y_final_resampled, verbose=0)


# --- 6. Оценка финальной модели на ОТДЕЛЬНОЙ ВАЛИДАЦИОННОЙ выборке для подбора порога ---
X_train_eval, X_val_eval, y_train_eval, y_val_eval = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\n--- Оценка финальной модели CatBoost на валидационной выборке для подбора порога ---")
y_pred_proba_val = final_catboost_model.predict_proba(X_val_eval)[:, 1]

best_f1_threshold = 0
best_threshold = 0.5
precision_at_best_threshold = 0
recall_at_best_threshold = 0

for threshold in np.arange(0.05, 0.96, 0.01):
    y_pred_tuned = (y_pred_proba_val >= threshold).astype(int)
    current_f1 = f1_score(y_val_eval, y_pred_tuned, zero_division=0)
    if current_f1 > best_f1_threshold:
        best_f1_threshold = current_f1
        best_threshold = threshold
        precision_at_best_threshold = precision_score(y_val_eval, y_pred_tuned, zero_division=0)
        recall_at_best_threshold = recall_score(y_val_eval, y_pred_tuned, zero_division=0)

print(f"Оптимальный порог для F1 на валидации: {best_threshold:.2f}")
print(f"Итоговая F1-мера на валидации (с оптимальным порогом): {best_f1_threshold:.4f}")
print(f"Precision при оптимальном пороге: {precision_at_best_threshold:.4f}")
print(f"Recall при оптимальном пороге: {recall_at_best_threshold:.4f}")


# --- 7. Предсказания для submission ---
print("\n--- Предсказания для submission ---")
test_preds_proba = final_catboost_model.predict_proba(X_test_final)[:, 1]

test_preds_tuned = (test_preds_proba >= best_threshold).astype(int)
submission['Churn'] = np.where(test_preds_tuned == 1, 'Yes', 'No')

print("\nПроверка структуры submission:")
print("Столбцы:", submission.columns.tolist())
print("Первые 5 строк перед сохранением:")
print(submission.head())

submission = submission[['id', 'Churn']]

submission.to_csv('submission_catboost_smoteenn_scaled_exp.csv', index=False)
print("Файл submission_catboost_smoteenn_scaled_exp.csv сохранен локально.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Размерность тренировочных данных: (5635, 20)
Размерность тестовых данных: (1408, 19)
Распределение классов в тренировочных данных:
 Churn
No     4138
Yes    1497
Name: count, dtype: int64

--- Запуск оптимизации гиперпараметров для CatBoost с Optuna ---

Лучшая попытка CatBoost (по Optuna):
  F1-мера (средняя по CV): 0.6350
  Лучшие гиперпараметры CatBoost:
    iterations: 1178
    depth: 5
    learning_rate: 0.042918381459893455
    l2_leaf_reg: 0.008885204316682235
    subsample: 0.6318025624463656
    colsample_bylevel: 0.9659878109903174
    min_data_in_leaf: 20
    border_count: 125
    early_stopping_rounds: 146
    scale_pos_weight: 1.2748406324050772
    sampling_strategy_smote: 0.6179516591453881
    n_neighbors_smote: 10

--- Обучение финальной модели CatBoost на всех сбалансированных тренировочных данных ---

--- Оценка финальной модели CatBoost на